In this notebook, we will use [TorchIO](http://torchio.rtfd.io/) and its new [`RSNAMICCAI`](https://torchio.readthedocs.io/datasets.html#rsnamiccai) dataset class to load, preprocess and write the challenge dataset.

In [ ]:
!pip install -q torchio==0.18.53

In [ ]:
from pathlib import Path
import torch
import torchio as tio
import matplotlib.pyplot as plt

plt.rcParams["figure.figsize"] = (12, 10)

In [ ]:
root_dir = '/kaggle/input/rsna-miccai-brain-tumor-radiogenomic-classification'
dataset = tio.datasets.RSNAMICCAI(root_dir)
len(dataset)

In [ ]:
subject = dataset[0]
subject

In [ ]:
print(subject.BraTS21ID)
print(subject.MGMT_value)
print(subject.T1w)
print(subject.T1wCE)
print(subject.T2w)
print(subject.FLAIR)

In [ ]:
subject.plot(reorient=False)

Results with these raw data will not be good because the images have different orientations, shapes and voxel sizes, i.e., they are in different voxel spaces. We need to preprocess the dataset to put images in the same voxel space. TorchIO to the rescue.

In [ ]:
preprocessing_transforms = (
    tio.ToCanonical(),
    tio.Resample(1, image_interpolation='bspline'),
    tio.Resample('T1w', image_interpolation='nearest'),
)
preprocess = tio.Compose(preprocessing_transforms)

We use [`ToCanonical`](https://torchio.readthedocs.io/transforms/preprocessing.html#tocanonical) to have a consistent orientation, [`Resample`](https://torchio.readthedocs.io/transforms/preprocessing.html#resample) the images to 1 mm isotropic spacing, and [`Resample`](https://torchio.readthedocs.io/transforms/preprocessing.html#resample) again to match the field of view of one of the images (the second resampling is lossless as images are already in the same voxel space).

Let's create datasets that apply our preprocessing transform after loading the images.

In [ ]:
train_set = tio.datasets.RSNAMICCAI(root_dir, train=True, transform=preprocess)
test_set = tio.datasets.RSNAMICCAI(root_dir, train=False, transform=preprocess)

To avoid resampling during training, which is computationally expensive, we preprocess our images now and save a new dataset to disk.

Use `demo=False` to preprocess all images in the dataset.

In [ ]:
def preprocess_dataset(dataset, out_dir, parallel=True, demo=True):
    import shutil
    import multiprocessing as mp
    from pathlib import Path
    from tqdm.notebook import tqdm
    if demo:  # just to showcase TorchIO
        dataset._subjects = dataset._subjects[:5]
    out_dir = Path(out_dir)
    labels_name = 'train_labels.csv'
    out_dir.mkdir(exist_ok=True, parents=True)
    shutil.copy(dataset.root_dir / labels_name, out_dir / labels_name)
    subjects_dir = out_dir / ('train' if dataset.train else 'test')
    if parallel:
        loader = torch.utils.data.DataLoader(
            dataset,
            num_workers=mp.cpu_count(),
            collate_fn=lambda x: x[0],
        )
        iterable = loader
    else:
        iterable = dataset
    for subject in tqdm(iterable):
        subject_dir = subjects_dir / subject.BraTS21ID
        for name, image in tqdm(subject.get_images_dict().items(), leave=False):
            image_dir = subject_dir / name
            image_dir.mkdir(exist_ok=True, parents=True)
            image_path = image_dir / f'{name}.nii.gz'
            image.save(image_path)

In [ ]:
out_dir = 'rsna-preprocessed'
if not Path(out_dir).is_dir():
    preprocess_dataset(train_set, out_dir)
    preprocess_dataset(test_set, out_dir)

In [ ]:
new_train_set = tio.datasets.RSNAMICCAI(out_dir, train=True)
new_test_set = tio.datasets.RSNAMICCAI(out_dir, train=False)

In [ ]:
subject = new_train_set[0]
subject.plot(reorient=False)

That's better! All images are now in the same voxel space.

Training can now be performed as usual, [combining TorchIO with other libraries like PyTorch Lightning and MONAI](https://colab.research.google.com/github/fepegar/torchio-notebooks/blob/main/notebooks/TorchIO_MONAI_PyTorch_Lightning.ipynb).